In [ ]:
# imports
import os
import sys
import pandas as pd
import numpy as np
import importlib
sys.path.insert(0, os.path.abspath(r'C:/Users/mmccann/repos/bonhoeffer/prey_capture/'))
import scipy.signal as ss
import scipy.stats as stat

import panel as pn
import holoviews as hv
from holoviews import opts, dim
from holoviews.operation import histogram
hv.extension('bokeh')
from bokeh.resources import INLINE

import paths
import processing_parameters
import functions_misc as fm
import functions_bondjango as bd
import snakemake_scripts.tc_calculate as tc
import snakemake_scripts.wf_tc_calculate as wf_tc
import functions_plotting as fp
import functions_data_handling as fdh
from functions_tuning import calculate_dff

In [ ]:
importlib.reload(processing_parameters)
# get the search query
search_string = processing_parameters.search_string + ', analysis_type:preprocessing'
parsed_search_string = fdh.parse_search_string(search_string)

# get the paths from the database
all_path = bd.query_database('analyzed_data', search_string)
input_path = [el['analysis_path'] for el in all_path if ('_preproc' in el['slug']) and (parsed_search_string['mouse'].lower() in el['slug'])]
# get the day, animal and rig
day = '_'.join(all_path[0]['slug'].split('_')[0:3])
rig = all_path[0]['rig']
animal = all_path[0]['slug'].split('_')[3:6]
animal = '_'.join([animal[0].upper()] + animal[1:])

# assemble the output path
out_path = os.path.join(paths.analysis_path, '_'.join((day, animal, rig, 'tcday.hdf5')))

# allocate memory for the data
raw_data = []
# allocate memory for excluded trials
excluded_trials = []
# for all the files
for files in input_path:
    # load the data
    with pd.HDFStore(files, mode='r') as h:
        if ('/matched_calcium' in h.keys()):

            # concatenate the latents
            dataframe = h['matched_calcium']
            # store
            raw_data.append(dataframe)
        else:
            excluded_trials.append(files)
print(f'Number of files loaded: {len(raw_data)}')

In [ ]:
tcs_dict, tcs_counts_dict, tcs_bins_dict = wf_tc.calculate_kinematic_tuning(raw_data[0], day, animal, rig)